In [ ]:
import pandas as pd

# Load att df

In [ ]:
playa_att_csv_path = '../data/playa_nogeometry.csv'
playa_att_df = pd.read_csv(playa_att_csv_path).set_index('id')

# Preprocess data to split up by state and county

In [ ]:
inun_csv_path = '../data/jrc-water_1984-2019.csv'

In [ ]:
inun_df = pd.read_csv(inun_csv_path).set_index('id').drop('.geo', axis=1).rename(columns={'mean':'inundation'})

In [ ]:
# Save state and county csvs
for state in playa_att_df['state'].unique():
    
    # Join state and county
    state_df = inun_df.join(playa_att_df.loc[playa_att_df['state']==state,
                                             ['state', 'countyfips', 'acres']],how='inner')
    
    # Split up weird index into year + months
    split_index = state_df['system:index'].str.split('_', expand=True)
    state_df = state_df.assign(year=split_index[0].astype('int16'),
                               month=split_index[1].astype('int16'),
                               system_index=split_index[2],
                               area=state_df['acres']*state_df['inundation'])
    state_df = state_df.drop('system:index', axis=1)
    
    # Save state csv
    state_out_path = '../data/state_county_csvs/inun_frac_{}.csv'.format(state)
    state_df.to_csv(state_out_path)
    
    # Save county csvs
    for county in state_df['countyfips'].unique():
        county_df = state_df.loc[state_df['countyfips']==county]
        county_out_path = '../data/state_county_csvs/counties/inun_frac_{}_{}.csv'.format(state, county)
        county_df.to_csv(county_out_path)

# Preprocess and split weather data

In [ ]:
weather_csv_path = '../data/era5_1984-2019.csv'

In [ ]:
weather_df = pd.read_csv(weather_csv_path).set_index('id').drop('.geo', axis=1)

In [ ]:
# Save state and county csvs
for state in playa_att_df['state'].unique():
    
    # Join state and county
    state_df = weather_df.join(playa_att_df.loc[playa_att_df['state']==state,
                                             ['state', 'countyfips', 'acres']],how='inner')
    
    # Split up weird index into year + months
    split_index = state_df['system:index'].str.split('_', expand=True)
    state_df = state_df.assign(year=split_index[0].str.slice(0,4).astype('int16'),
                               month=split_index[0].str.slice(4,6).astype('int16'),
                               system_index=split_index[1])
    state_df = state_df.drop('system:index', axis=1)
    
    # Save state csv
    state_out_path = '../data/state_county_csvs/weather_{}.csv'.format(state)
    state_df.to_csv(state_out_path)
    
    # Save county csvs
    for county in state_df['countyfips'].unique():
        county_df = state_df.loc[state_df['countyfips']==county]
        county_out_path = '../data/state_county_csvs/counties/weather_{}_{}.csv'.format(state, county)
        county_df.to_csv(county_out_path)